要运行此操作，请按“*Runtime*”，然后按“*运行*” ** free ** tesla Tesla T4 Google COLAB实例！
<div class =“ Align-Center”>
<a href =“ https://unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/raw/main/main/mains/unsloth%20new%20new%20logo.png.png”
<a href =“ https://discord.gg/unsloth”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/main/main/main/images/images/discord button.png button.png
<a href =“ https://docs.unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/unsloth/main/main/main/mains/images/documentation%20green%20green%20breen%20button.png?png?raw=raw=true width width” <i>在<a href =“ https://github.com/unslothai/unsloth”> github </a> </i>⭐中
</div>

要在您自己的计算机上安装不塞，请按照我们的github页面上的安装说明[here](https://docs.unsloth.ai/get-started/installing-+-updating)上的安装说明。

您将学习如何做[data prep](#Data)，如何[train](#Train)，如何[run the model](#Inference)，＆xx_markDown_link_xx


### 消息

** New ** Unsploth现在支持培训OpenAi的新** GPT-oss **模型！您可以通过我们的** [Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)免费启动Finetune GPT-oss **！

Unsploth现在支持文本对语音（TTS）模型。阅读我们的xx_markDown_link_xx。

阅读我们的** [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning) **并查看我们的新** [Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune) ** Quants优先量优于其他量化方法！

请访问我们的所有文档，以获取我们的所有XX_MarkDown_link_xx和[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)。


### 安装

In [ ]:
%%capture
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install unsloth
!pip install --upgrade transformers==4.53.2 "huggingface_hub>=0.34.0" "datasets>=3.4.1,<4.0.0"


### 不塞

In [ ]:
%env UNSLOTH_RETURN_LOGITS=1 # 将其运行以禁用CCE，因为它不支持CPT

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # 选择任何！我们在内部自动支持绳索扩展！
dtype = None # 没有自动检测。 Tesla T4，V100，Bfloat16的float16 for Ampere+
load_in_4bit = True # 使用4位量化来减少内存使用量。可以是错误的。

# 4位预量化模型我们支持4倍下载 + no ooms。
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # 新的Mistral V3 2X快速！
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3-15万亿代币2型快速！
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # PHI-3 2倍！
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2倍！
] # https://huggingface.co/unsloth的更多型号

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # 选择任何！例如Teknium/OpenHermes-2.5-Mistral-7b
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token =“ hf _...”，＃如果使用元模型，例如meta-llama/llama-2-7b-hf
)

现在，我们添加Lora适配器，因此我们只需要更新所有参数的1％至10％！

我们还添加了`embed_tokens'和`lm_head''，以允许模型从分发数据中学习。

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # 选择任何数字> 0！建议8、16、32、64、128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # 添加以持续预处理
    lora_alpha = 32,
    lora_dropout = 0, # 支持任何人，但= 0已优化
    bias = "none",    # 支持任何人，但是=“无”
    # [NEW]“ Unsploth”使用的VRAM减少了30％，适合2倍批量的尺寸！
    use_gradient_checkpointing = "unsloth", # 在很长的背景下真实或“不整齐”
    random_state = 3407,
    use_rslora = True,   # 我们支持排名稳定的洛拉
    loftq_config = None, # 和Loftq
)

<a name =“ data”> </a>
###数据准备
现在，我们使用XX_MarkDown_link_xx的韩国子集首先不断预认识该模型。您可以使用**您喜欢的任何语言**！转到[Wikipedia dataset](https://huggingface.co/datasets/wikimedia/wikipedia)查找自己的语言！

** [注意] **仅训练完成（忽略用户的输入）读取TRL的文档[Wikipedia's List of Languages](https://en.wikipedia.org/wiki/List_of_Wikipedias)。

** [注意] **请记住将** eos_token **添加到令牌化输出中！否则，您将获得无限的几代人！

如果您想将`llama-3`模板用于sharegpt数据集，请尝试我们的对话[here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only)-conversations.ipynb）

对于诸如新颖写作之类的文本完成，请尝试此[notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_v0.3_(7B)-text_completion.ipynb）。

** [注意] **使用https://translate.google.com从英语翻译成韩文！

In [ ]:
# Wikipedia提供了标题和文章文本。
# 使用https://translate.google.com！
_wikipedia_prompt = """Wikipedia Article
# ＃＃ 标题： {}

# ＃＃ 文章：
{}"""
# 变成：
wikipedia_prompt = """위키피디아 기사
# ＃＃ 标题： {}

# ＃＃ 文章：
{}"""

EOS_TOKEN = tokenizer.eos_token # 必须添加eos_token
def formatting_prompts_func(examples):
    titles = examples["title"]
    texts  = examples["text"]
    outputs = []
    for title, text in zip(titles, texts):
        # 必须添加eos_token，否则您这一代将永远持续下去！
        text = wikipedia_prompt.format(title, text) + EOS_TOKEN
        outputs.append(text)
    return { "text" : outputs, }
pass

我们只使用1％的数据集来加快速度！用于长期运行！

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikimedia/wikipedia", "20231101.ko", split = "train",)

# 我们选择1％的数据以使培训更快！
dataset = dataset.train_test_split(train_size = 0.01)["train"]

dataset = dataset.map(formatting_prompts_func, batched = True,)

<a name =“ train”> </a>
###继续预处理
现在，让我们使用Unsploth的“ unsplostrainer”！这里更多文档：[TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)。我们执行20个步骤来加快速度，但是您可以将`num_train_epochs = 1`设置为完整运行，然后关闭`max_steps = none“。

还将`embedding_learning_rate`设置为学习率至少比“ Learning_rate”小2倍或10倍，以进行持续的预处理！

In [ ]:
from transformers import TrainingArguments
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        # 使用Hallup_ratio和num_train_epochs进行较长的运行！
        max_steps = 120,
        warmup_steps = 10,
        # 热身= 0.1，
        # num_train_epochs = 1，

        # 为嵌入矩阵选择2至10倍的学习率！
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,

        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # 将其用于Wandb等
    ),
)

In [ ]:
# @Title显示当前内存统计
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

### 指令填充

现在，我们使用xx_markDown_link_xx，但在韩文中进行了翻译！

对于羊驼或XX_Markdown_link_xx的原始GPT4数据集，请访问XX_Markdown_link_xx，以获取羊Alpaca数据集的其他翻译。

In [ ]:
from datasets import load_dataset

alpaca_dataset = load_dataset("FreedomIntelligence/alpaca-gpt4-korean", split="train")

我们打印1个示例：

In [ ]:
print(alpaca_dataset[0])

我们再次使用https://translate.google.com/将羊驼格式翻译成韩文

In [ ]:
_alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

# ＃＃ 操作说明：
{}

# ＃＃ 回复：
{}"""
# 变成：
alpaca_prompt = """다음은 작업을 설명하는 명령입니다. 요청을 적절하게 완료하는 응답을 작성하세요.

# ##指南：
{}

# ＃＃ 回复：
{}"""

EOS_TOKEN = tokenizer.eos_token # 必须添加eos_token
def formatting_prompts_func(conversations):
    texts = []
    conversations = conversations["conversations"]
    for convo in conversations:
        # 必须添加eos_token，否则您这一代将永远持续下去！
        text = alpaca_prompt.format(convo[0]["value"], convo[1]["value"]) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

alpaca_dataset = alpaca_dataset.map(formatting_prompts_func, batched = True,)

我们再次采用“不舒服”并进行指导填充！

In [ ]:
from transformers import TrainingArguments
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = alpaca_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        # 使用num_train_epochs和Hallup_ratio进行长期运行！
        max_steps = 120,
        warmup_steps = 10,
        # 热身= 0.1，
        # num_train_epochs = 1，

        # 为嵌入矩阵选择2至10倍的学习率！
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,

        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # 将其用于Wandb等
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @Title显示最终内存和时间统计
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name =“推理”> </a>
###推理
让我们运行模型！您可以更改指令和输入 - 将输出空白保留！

切记使用https://translate.google.com/！

In [ ]:
# 羊Alpaca_prompt =从上方复制
FastLanguageModel.for_inference(model) # 启用本地2倍更快的推理
inputs = tokenizer(
[
    alpaca_prompt.format(
        # “继续斐波那契序列：1、1、2、3、5、8，”，＃指令
        "피보나치 수열을 계속하세요: 1, 1, 2, 3, 5, 8,", # 操作说明
        "", # 输出 - 将此空白留给生成！
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

您还可以使用“ TextStreamer”进行连续推理 - 因此，您可以通过令牌看到代币，而不是一直在等待！

In [ ]:
# 羊Alpaca_prompt =从上方复制
FastLanguageModel.for_inference(model) # 启用本地2倍更快的推理
inputs = tokenizer(
[
    alpaca_prompt.format(
        # “韩国音乐是什么样的？”
        "한국음악은 어떤가요?", # 操作说明
        "", # 输出 - 将此空白留给生成！
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

通过使用https://translate.google.com/我们得到
```
Korean music is classified into many types of music genres.

This genre is classified into different music genres such as pop songs,

rock songs, classical songs and pop songs, music groups consisting of drums, fans, instruments and singers
```

<a name =“ save”> </a>
###保存，加载固定模型
要将最终模型保存为Lora适配器，请使用HuggingFace的“ push_to_hub”进行在线保存或`save_pretaining'用于本地保存。

** [注意] **这仅保存洛拉适配器，而不是完整的模型。要节省16位或GGUF，请向下滚动！

In [ ]:
model.save_pretrained("lora_model")  # 本地节省
tokenizer.save_pretrained("lora_model")
# model.push_to_hub（“ your_name/lora_model”，token =“ ...”）＃在线保存
# tokenizer.push_to_hub（“ your_name/lora_model”，token =“ ...”）＃在线保存

现在，如果您想加载洛拉适配器，我们刚刚保存用于推理，请将`false``设置为true`：：true'：

In [ ]:
if False:
    from unsloth import FastLanguageModel

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="lora_model",  # 您用于培训的模型
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # 启用本地2倍更快的推理

# 羊Alpaca_prompt =您必须从上方复制！

inputs = tokenizer(
    [
        alpaca_prompt.format(
            # “广泛描述地球。”，＃指示
            "지구를 광범위하게 설명하세요.",
            "",  # 输出 - 将此空白留给生成！
        ),
    ],
    return_tensors="pt",
).to("cuda")


from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs, streamer=text_streamer, max_new_tokens=128, repetition_penalty=0.1
)

通过使用https://translate.google.com/我们得到
```
Earth refers to all things including natural disasters such as local derailment

and local depletion that occur in one space along with the suppression of water, gases, and living things.

Most of the Earth's water comes from oceans, atmospheric water, underground water layers, and rivers and rivers.
```

yikes语言模型有点烦恼！改变温度并使用采样肯定会使输出变得更好！

您也可以使用拥抱脸的“ AutomodelForpeftCausAllm”。仅在未安装``'''''时就使用。由于不支持“ 4bit”模型下载，因此它可以慢慢放慢，并且不舒服的**推理快2倍**。

In [ ]:
if False:
    # 我高度建议 - 如果可能的话，请使用不塞
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer

    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",  # 您用于培训的模型
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### 节省float16 vllm

我们还直接支持保存到`float16`。为float16选择`merged_16bit`或用于int4的merged_4bit`。我们还允许`Lora`适配器作为后备。使用`push_to_hub_merged`上传到您的拥抱脸部帐户！您可以访问https://huggingface.co/settings/tokens for个人令牌。

In [ ]:
# 合并为16位
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# 合并到4位
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# 只是洛拉适配器
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / LLAMA.CPP转换
要保存到`gguf' /`llama.cpp`，我们现在在本地支持它！我们克隆“ llama.cpp”，默认将其保存到Q8_0`。我们允许所有方法'q4_k_m`。使用`save_pretained_gguf`进行本地保存，然后`push_to_hub_gguf`将上传到HF。

一些支持的量化方法（我们的[Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)上的完整列表）：
*`Q8_0`-快速转换。大量资源使用，但通常可以接受。
*`q4_k_m`-推荐。使用q6_k作为注意到的一半。
*`q5_k_m`-推荐。使用q6_k进行一半的注意。wv和feed_forward.w2张量，else q5_k。

In [ ]:
# 保存到8位Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# 保存至16位GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# 保存到Q4_K_M GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q5_k_m", token = "")

现在，在llama.cpp中使用`model-unsloth.gguf`文件或`model-unsloth-q4_k_m.gguf`文件或基于UI的系统（例如JAN或OPEN webUI）。您可以安装JAN XX_MARKDOWN_LINK_XX并打开webui [here](https://github.com/janhq/jan)

我们完成了！如果您对不舒服有任何疑问，我们有一个XX_MarkDown_link_xx频道！如果您发现任何错误或想对最新的LLM内容进行更新，或者需要帮助，加入项目等，请随时加入我们的不和谐！

其他一些链接：
1。训练您自己的推理模型-Llama grpo笔记本XX_Markdown_link_xx-grpo.ipynb）
2。将芬太尼保存到奥尔马。 [here](https://github.com/open-webui/open-webui)-ollama.ipynb）
3。Llama3.2视觉燃烧 -  X射线照相用例。 [Discord](https://discord.gg/unsloth)-vision.ipynb）
6。请参见DPO，ORPO，继续预处理，对话finetuning等笔记本，以及我们的xx_markDown_link_xx上的更多内容！

<div class =“ Align-Center”>
  <a href =“ https://unsloth.ai”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/raw/main/main/images/unsloth%20new%20new%20logo.png”
  <a href =“ https://discord.gg/unsloth”> <img src =“ https://github.com/unslothai/unsloth/unsloth/raw/main/main/main/images/images/discord.png，png
  <a href =“ https://docs.unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/unsloth/main/main/main/images/images/images/documentation%20green%20green%20breen%20button.png?png?png?raw=true width =”

  如果您需要帮助，请加入DISCORD +⭐️<i>在<a href =“ https://github.com/unslothai/unsloth上”
</div>
